# Goal - Understand what power plants are Net Generators and where they are located
* What is the plant energy resource make up of each state

[EIA Vocab](https://www.eia.gov/tools/glossary/index.php?id=G)

# Getting Powerplant Data

# Probably can query this instead of downloading each link below
# https://www.eia.gov/opendata/qb.php?category=1017

* [BioMass](https://atlas.eia.gov/datasets/biomass-2/explore?location=44.619557%2C61.504001%2C3.67&showTable=true)
* [Petroleum](https://atlas.eia.gov/datasets/petroleum-2/explore?location=44.619557%2C61.504001%2C3.67)
* [Other](https://atlas.eia.gov/datasets/other/explore?location=44.619557%2C61.504001%2C3.67)
* [Battery Storage](https://atlas.eia.gov/datasets/battery-storage/explore?location=44.619557%2C61.504001%2C3.67)
* [Hydro Electric](https://atlas.eia.gov/datasets/hydroelectric-2/explore?location=44.619557%2C61.504001%2C3.67)
* [Nuclear](https://atlas.eia.gov/datasets/nuclear/explore?location=44.619557%2C61.504001%2C3.67)
* [Solar](https://atlas.eia.gov/datasets/solar-2/explore?location=44.619557%2C61.504001%2C3.67)
* [Wind](https://atlas.eia.gov/datasets/wind-2/explore?location=44.619557%2C61.504001%2C3.67)
* [Coal](https://atlas.eia.gov/datasets/coal-1/explore?location=44.619557%2C61.504001%2C3.67)
* [Natural Gas](https://atlas.eia.gov/datasets/natural-gas-1/explore?location=44.619557%2C61.504001%2C3.67)
* [Geothermal](https://atlas.eia.gov/datasets/geothermal-1/explore?location=44.619557%2C61.504001%2C3.67)
* [Pumped Storage](https://atlas.eia.gov/datasets/pumped-storage/explore?location=44.619557%2C61.504001%2C3.67)

* [Map](https://atlas.eia.gov/maps/power-plants-1/about)

# Setup

In [2]:
try:
    from google.colab import drive

    drive.mount('./drive/')

    %cd drive/Shareddrives/Data606_Energy

except:
    print("No Colab Environment")

Mounted at ./drive/
/content/drive/Shareddrives/Data606_Energy


In [3]:
#Data Oriented
import pandas as pd
import numpy as np



#General Purpose
import requests 
import glob
import json
import sys

#Notebook progress bar
from tqdm.autonotebook import tqdm

#Created
sys.path.append("helpers/")

from energygrid import EGRID

from helper_functions import write_csv,combine_like_files,clean_eia_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [8]:
energy_grid = EGRID()
energy_grid.get_states()
energy_grid.get_plant_fuel_types()

# High Level Powerplants

In [4]:
df_powerplants = combine_like_files("data/powerplant/*_Power_Plants.csv",pd.read_csv)


df_powerplants

,X,Y,OBJECTID,Code,Power_Plant_Name,Utility_Name,Utility_ID,Sector_Name,City,County,State,Zip,Street_Address,PrimSource,Install_MW,Total_MW,Coal_MW,NG_MW,Crude_MW,Hydro_MW,HydroPS_MW,Nuclear_MW,Solar_MW,Wind_MW,Source_Desc,Technology_Desc,Data_Source,Period,Latitude,Longitude,Geoth_MW,Battery_MW,Others_MW,Biomass_MW
0,-118.111059,48.620336,1,550,Kettle Falls Generating Station,Avista Corp,20169,Electric Utility,Kettle Falls,Stevens,Washington,99141,1151 Hyw. 395 N.,biomass,57.9,56.2,0.0,6.2,0.0,0.0,0.0,0.0,0.0,0.0,"Biomass = 50 MW, Natural Gas = 6.2 MW",Wood/Wood Waste Biomass; Natural Gas Fired Com...,"EIA-860, EIA-860M and EIA-923",202001,48.620336,-118.111059,0.0,0.0,0.0,50.0
1,-73.208056,44.491700,2,589,J C McNeil,City of Burlington Electric - (VT),2548,Electric Utility,Burlington,Chittenden,Vermont,5401,111 Intervale Road,biomass,59.5,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biomass = 52 MW,Wood/Wood Waste Biomass,"EIA-860, EIA-860M and EIA-923",202001,44.491700,-73.208056,0.0,0.0,0.0,52.0
2,-88.455800,46.755300,3,1772,John H Warden,L'Anse Warden Electric Company LLC,56133,IPP Non-CHP,L'Anse,Baraga,Michigan,49946,157 S Main St,biomass,18.7,17.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biomass = 17.7 MW,Wood/Wood Waste Biomass,"EIA-860, EIA-860M and EIA-923",202001,46.755300,-88.455800,0.0,0.0,0.0,17.7
3,-92.151711,46.735331,4,1897,M L Hibbard,"ALLETE, Inc.",12647,Commercial CHP,Duluth,St Louis,Minnesota,55807,4913 Main Street,biomass,72.8,59.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biomass = 59.8 MW,Wood/Wood Waste Biomass,"EIA-860, EIA-860M and EIA-923",202001,46.735331,-92.151711,0.0,0.0,0.0,59.8
4,-92.516912,44.569200,5,1926,Red Wing,Northern States Power Co - Minnesota,13781,Electric Utility,Red Wing,Goodhue,Minnesota,55901,801 5th st. East,biomass,23.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biomass = 18 MW,Municipal Solid Waste,"EIA-860, EIA-860M and EIA-923",202001,44.569200,-92.516912,0.0,0.0,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9763,-119.469986,36.585794,3269,63514,Moonlight Packing - Phase 2,CalCom Energy,63226,IPP Non-CHP,NaN,NaN,California,0,NaN,solar,1.4,1.4,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,Solar = 1.4 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202001,36.585794,-119.469986,0.0,0.0,0.0,0.0
9764,-117.981000,35.157000,3270,63516,California City,GSRP,61944,IPP Non-CHP,NaN,NaN,California,0,NaN,solar,1.9,1.9,0.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,Solar = 1.9 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202001,35.157000,-117.981000,0.0,0.0,0.0,0.0
9765,-77.825430,39.621430,3271,63523,Spruce - WCMD - Rubble II,GSRP,61944,IPP Non-CHP,NaN,NaN,Maryland,0,NaN,solar,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Solar = 2 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202001,39.621430,-77.825430,0.0,0.0,0.0,0.0
9766,-77.823320,39.618490,3272,63524,Spruce - WCMD - Rubble I,GSRP,61944,IPP Non-CHP,NaN,NaN,Maryland,0,NaN,solar,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Solar = 2 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202001,39.618490,-77.823320,0.0,0.0,0.0,0.0


# Investigate Net Generation - https://www.eia.gov/opendata/qb.php?category=1017

* **The endpoint below pulls all energy plants. But need to identifiy**
* ELEC.PLANT.GEN.**{Plant-NG-GT}**.M
    * Plant - Plant Code
    * NG - Natural Gas

## Plants

In [7]:
# Get list of plants
plants = df_powerplants["Code"].unique().tolist()
print("Example Plants:",plants[:10])

Example Plants: [550, 589, 1772, 1897, 1926, 1934, 2018, 7413, 7690, 7701]


## Fuel Types

In [8]:
# Fuel type dictionaries
fuel_types_1_dict = {
                "hydroelectric":"WAT",
                "bituminous coal": "BIT",
                "subbituminous coal":"SUB",
                "black liquour":"BLQ",
                "tire-derived fuels":"TDF",
                "petroleum coke": "PC",
                "synthetic coal":"SC",
                "biogenic municipal solid waste":"MSB",
                "Disolate Fue Oil":"DFO",
                "Other":"OTH",
                "landfill gas":"LFG",
                "gaseous propane":"PG",
                "waste/other coal":"WC",
                "waste/other oil":"WO",
                "other gas":"OG",
                "kerosene":"KER",
                "residual fuel oil":"RFO",
                "lignite coal":"LIG",
                            
                "other biomass solids":"OBS",
                "jet fuel":"JF",

                "coal-derived synthetic gas":"SGC",
                "blast furnace gas": "BFG",
                "Wind":"WND",
                "Solar": "SUN",
                "Nuclear":"NUC",
                "agricultural by-products":"AB",
                "sludge waste":"SLW",
                "purchased steam":"PUR",
                "other biomass gas":"OBG",
                "subbituminous coal":"SUB",
                "geothermal":"GEO",
                "batteries or other use of electricity as an energy source":"MWH",
                "waste heat": "WH","non-biogenic municipal solid waste":"MSN",
                "wood/wood waste solids":"WDS",
                "municipal solid waste":"MSW",
                "Natural Gas":"NG",
                "biogenic municipal solid waste":"MSB",
                "refined coal":"RC",

}

fuel_types_2_dict = {

                }




fuel_type_list = list(fuel_types_2_dict.values())
print("Look up Fuel Types:",fuel_type_list)

Look up Fuel Types: []


In [9]:
# Total amount of fuel types searched 
# There are probably a few missing, but these are the major ones
print("Total amount of fuel types searched.") 
len(fuel_types_1_dict.keys())

Total amount of fuel types searched.


37

In [10]:
# Search in fuel type lists
lookup_fuel_type = list(fuel_types_1_dict.values())
[source for source in lookup_fuel_type if source == "WH"]

['WH']

# Gather EIA data on Plant Fuel Type Net Generation

In [42]:
unique_plants_found = set(df_plant_fuel_level["plant_code"].to_list())
unique_plants = set(df_powerplants["Code"].to_list())

unique_plants_left = list(unique_plants.difference(unique_plants_found))

print("Percentage of Plants Found:","{0:.2%}".format(len(unique_plants_found)/len(unique_plants)),
      "Percentage of Plants Not Found:","{0:.2%}".format(len(unique_plants_left)/len(unique_plants)),"\n"
      "Total Plants:",len(unique_plants), "\n"
      "Plants Found:",len(unique_plants_found), "\n"
      "Plants Still Outstanding:",len(unique_plants_left), "\n"
        )

      

Percentage of Plants Found: 98.38% Percentage of Plants Not Found: 1.62% 
Total Plants: 9768 
Plants Found: 9610 
Plants Still Outstanding: 158 



In [43]:
np.random.choice(unique_plants_left,56,replace=False)

array([63145, 63149, 61233, 63412, 60719, 63137, 50952, 63177, 63423,
       62647, 61235, 50363, 63461, 61083, 62695, 63346, 62852, 60328,
       50337, 63150, 62568, 61036, 61273, 63063, 61808, 63035, 61148,
       60831, 63450, 61014, 63022, 61147, 62958, 61639, 63042, 50942,
       61474, 61064, 61034, 61150, 60429, 58190, 63013, 63161, 62782,
       61482, 63497, 63000, 62477, 61739, 62154, 61146, 63373, 63143,
       62728,  2475])

In [44]:
pd.DataFrame({"plant_codes_not_found":unique_plants_left}).merge(df_powerplants,left_on="plant_codes_not_found",right_on="Code",how="left").describe()

,plant_codes_not_found,X,Y,OBJECTID,Code,Utility_ID,Zip,Install_MW,Total_MW,Coal_MW,NG_MW,Crude_MW,Hydro_MW,HydroPS_MW,Nuclear_MW,Solar_MW,Wind_MW,Period,Latitude,Longitude,Geoth_MW,Battery_MW,Others_MW,Biomass_MW
count,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.0,158.0,158.000000,158.000000,158.0,158.000000,158.000000,158.0,158.000000,158.000000,158.000000
mean,60412.803797,-82.267140,33.726549,1919.664557,60412.803797,50548.512658,26937.563291,44.395570,40.705063,2.875949,10.087975,23.325949,0.634810,0.0,0.0,2.332278,0.631646,202001.0,33.726549,-82.267140,0.0,0.312025,0.031646,0.472785
std,8584.537762,17.802430,10.901045,1086.314171,8584.537762,19802.577051,32412.376496,180.377964,166.475573,36.150123,76.908377,139.363550,3.615077,0.0,0.0,4.903414,6.238631,0.0,10.901045,17.802430,0.0,1.915454,0.397779,4.201405
min,491.000000,-158.058100,17.947115,13.000000,491.000000,803.000000,0.000000,0.900000,0.800000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,202001.0,17.947115,-158.058100,0.0,0.000000,0.000000,0.000000
25%,61132.250000,-93.182456,18.431825,823.250000,61132.250000,56990.000000,738.000000,1.500000,1.425000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,202001.0,18.431825,-93.182456,0.0,0.000000,0.000000,0.000000
50%,61954.500000,-76.055141,37.784688,2050.500000,61954.500000,60693.000000,11060.500000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,202001.0,37.784688,-76.055141,0.0,0.000000,0.000000,0.000000
75%,63031.000000,-66.736354,42.267725,3088.750000,63031.000000,61610.000000,55947.000000,8.375000,8.375000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.650000,0.000000,202001.0,42.267725,-66.736354,0.0,0.000000,0.000000,0.000000
max,63498.000000,-65.279560,60.540470,3265.000000,63498.000000,63246.000000,99574.000000,1534.000000,1461.200000,454.400000,765.600000,1461.200000,34.000000,0.0,0.0,45.000000,75.000000,202001.0,60.540470,-65.279560,0.0,21.800000,5.000000,50.400000


# Get Data

In [5]:
# Load Credentials
with open("creds/eia_creds.json","r") as f:

    key = json.load(f)["key"]

In [30]:
plant_kickouts = set() #These are plants that are not found


# Look into whether these plants are also petrolum 
# Merge them on the plants above to see
def get_plant_net_generation(plants,fuel_types):

    print("Fuel Types:",len(fuel_types))

    df_master = pd.DataFrame()


    for fuel_type in fuel_types:

        print(fuel_type)

        for index, plant in enumerate(tqdm(plants)):
            try:
                response = requests.get(f"http://api.eia.gov/series/?api_key={key}&series_id=ELEC.PLANT.GEN.{plant}-{fuel_type}-ALL.M").json()

                data_series = response.get("series")[0]

                df = pd.DataFrame(data_series)

                df_clean = clean_eia_df(df,"mWh")

                df_clean["plant_code"] = plant

                df_clean["fuel_type"] = fuel_type

                df_master = pd.concat([df_master,df_clean])



            except TypeError:
                plant_kickouts.add((index,plant,fuel_type))
                continue

        #fn = f"data/powerplant/plants_by_state/Net_Generation/Fuel_Type/combine/second/{fuel_type}_net_generation.json"

        #df_master.to_json(fn,orient="records")

    return df_master
        


#index and plants combinations not found
#write_csv("data/powerplant/plants_by_state/Net_Generation/sample_kickouts.csv",["index","plant_code","fuel_type"],plant_kickouts)

In [31]:
fuel_types = list(energy_grid.plant_fuel_types.values())

In [32]:
df_check = get_plant_net_generation(["876"],fuel_types)

Fuel Types: 37
WAT


  0%|          | 0/1 [00:00<?, ?it/s]

BIT


  0%|          | 0/1 [00:00<?, ?it/s]

SUB


...

  0%|          | 0/1 [00:00<?, ?it/s]

NG


  0%|          | 0/1 [00:00<?, ?it/s]

RC


  0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
df_check.query('date < "2001-12-01"')

,series_id,name,f,description,source,lat,lon,latlon,updated,country,state,mWh,date,year,plant_code,fuel_type
37,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,0.0,2001-11-01,2001,876,DFO
38,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,0.0,2001-10-01,2001,876,DFO
39,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,0.0,2001-09-01,2001,876,DFO
40,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,0.0,2001-08-01,2001,876,DFO
41,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,0.0,2001-07-01,2001,876,DFO
42,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,0.0,2001-06-01,2001,876,DFO
43,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,0.0,2001-05-01,2001,876,DFO
44,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,4345.0,2001-04-01,2001,876,DFO
45,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,0.0,2001-03-01,2001,876,DFO
46,ELEC.PLANT.GEN.876-DFO-ALL.M,Net generation : Kincaid Generation LLC (876) ...,M,NaN,"EIA, U.S. Energy Information Administration",39.590556,-89.496389,"39.590556,-89.496389",2016-07-07T17:18:42-0400,USA,IL,0.0,2001-02-01,2001,876,DFO


# Plant Kickouts - Plants that do not have data for fuel type

In [ ]:
df_kickouts = (pd.read_csv("data/powerplant/plants_by_state/Net_Generation/sample_kickouts.csv")
.merge(df_powerplants,left_on="plant_code",right_on="Code",how="left")
)

df_kickouts.query('plant_code == 57842')

In [ ]:
df_kickouts = (pd.read_csv("data/powerplant/plants_by_state/Net_Generation/sample_kickouts.csv")
.merge(df_powerplants,left_on="plant_code",right_on="Code",how="left")
)["plant_code"].unique().shape